In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = "data/train.csv"
test = "data/test.csv"


train_df = pd.read_csv(train, index_col='Id')
test_df = pd.read_csv(test, index_col='Id')

all_data = pd.concat((train_df, test_df), sort=False)

#print(all_data.head(10))

#print (all_data.columns)

/Users/dgangwar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [3]:
# Check if any column contains a NaN value
#print(df.isnull().any())

# Provides a NaN count for each column.
#print(df.isnull().sum())

# Checking % data is null set in the data columns
null_col = (all_data.isnull().sum()/len(all_data)) * 100
null_col = null_col.sort_values(ascending=False)
print(null_col[:20])

# Remove column if all value in it are NaN.
#df = df.dropna(axis=1, how='all')

# Remove row if all value in it are NaN.
#df = df.dropna(axis=0, how='all')

#df = df.isnull().sum()
#print (df)

PoolQC          99.657417
MiscFeature     96.402878
Alley           93.216855
Fence           80.438506
SalePrice       49.982871
FireplaceQu     48.646797
LotFrontage     16.649538
GarageQual       5.447071
GarageYrBlt      5.447071
GarageFinish     5.447071
GarageCond       5.447071
GarageType       5.378554
BsmtCond         2.809181
BsmtExposure     2.809181
BsmtQual         2.774923
BsmtFinType2     2.740665
BsmtFinType1     2.706406
MasVnrType       0.822199
MasVnrArea       0.787941
MSZoning         0.137033
dtype: float64


In [4]:
# Dropping columns which has mostly all data cell as Null
all_data.drop(columns= {'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'}, inplace=True)
print(all_data.columns)

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BldgType',
       'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtQual', 'BsmtUnfSF',
       'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'EnclosedPorch',
       'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fireplaces',
       'Foundation', 'FullBath', 'Functional', 'GarageArea', 'GarageCars',
       'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt',
       'GrLivArea', 'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle',
       'KitchenAbvGr', 'KitchenQual', 'LandContour', 'LandSlope', 'LotArea',
       'LotConfig', 'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass',
       'MSZoning', 'MasVnrArea', 'MasVnrType', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SalePrice', 'S